In [1]:
#--------- Python Libraries ---------#

import requests
import json
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import date
import datetime
import dateutil.parser

import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

import string
import Levenshtein as lev

SPOTIPY_CLIENT_ID = 'a5ad437675474f19bc6c76bb096dee85'
SPOTIPY_CLIENT_SECRET = '69b402da629b41d498354ae80444c3d9'

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))


In [22]:
#--------- Web crawling for URLs ---------#


list_of_URLs = []

#--------- Web crawling for URLs ---------#

source = requests.get('https://www.theguardian.com/tone/livereview?page=2').text
soup = BeautifulSoup(source, 'lxml')
     
for link in soup.find_all('a',{'data-link-name' : 'article'}):
    URL = link.get('href')
    list_of_URLs.append(URL)

# Remove duplicate URLs in list
list_of_URLs = list(set(list_of_URLs))
print(list_of_URLs)

['https://www.theguardian.com/music/2020/jun/10/paul-lewis-review-beethoven-and-schubert-wigmore-hall-radio-3', 'https://www.theguardian.com/music/2020/apr/26/the-metropolitan-opera-at-home-gala-review', 'https://www.theguardian.com/music/2020/jun/04/nicholas-daniel-julius-drake-review-wigmore-hall-radio-3', 'https://www.theguardian.com/music/2020/jun/15/imogen-cooper-review-wigmore-hall-bbc-radio-3', 'https://www.theguardian.com/music/2020/mar/16/bbcso-edward-gardner-review-barbican-london-schoenberg-brahms-viennese-romanticism', 'https://www.theguardian.com/music/2020/jun/05/pavel-kolesnikov-and-samson-tsoy-wigmore-hall-radio-3-review-piano-duo-bring-electrifying-intimacy', 'https://www.theguardian.com/music/2020/mar/16/jon-hopkins-review-brighton-dome', 'https://www.theguardian.com/music/2020/mar/14/giulio-cesare-julius-ceasar-english-touring-opera-review-susanna-london-handel-festival-linbury', 'https://www.theguardian.com/music/2020/jun/01/stephen-hough-live-concert-wigore-hall-bb

In [27]:

# Create empty dictionary
critic_review_dict = {} 
critic_review_dict['concerts'] = []

#--------- Loop through list of URLs ---------#
 
for i in list_of_URLs:
    
    # Reset values
    critic_name = 'The Guardian'
    author = ''
    url = ''
    headline = ''
    text = ''
    rating = 0
    max_rating = 5
    artist_name = ''
    venue_name = ''
    publication_date = ''
    lang = 'en'
    genre = []
    venue_place = ''
    concert_date = ''
    
    url = i
    
    
    #--------- API Request ---------#
    
    # Take the ID part of the URL
    url = i.split('https://www.theguardian.com/')[-1]
    
    # Request content from URL
    api_key = '&api-key=bfd08b7e-fee2-4109-a686-2637bddfdc70'
    url_item = url
    url_api = 'https://content.guardianapis.com/'+ url_item + '?'
    show_fields = 'show-fields=headline,starRating,trailText,showInRelatedContent,id,byline,lang,firstPublicationDate,body'
    show_references = 'show-references=musicbrainzgenre'
    shouldHideAdverts = 'shouldHideAdverts=true'
    show_tags = 'show-tags=all'
    response = requests.get(url_api + show_fields + '&' + show_references + '&' + shouldHideAdverts + '&' + show_tags + api_key)

    print('Status Code:', response.status_code)
    response = response.json()
    
    
    
    # Dictionary Navigate
    try:
        fields = response.get("response").get('content').get('fields')
    except:
        'Could not load review'
        continue
    
    #--------- Check Genres ---------#
    
    try: 
        get_genres = response.get("response").get('content').get('references')
        print('Genre extracted from API')
    
    except:
        print('No genre extracted from API')


    try:
        for i in get_genres:
            temp_genre = i.get('id')
            temp_genre = temp_genre.split('musicbrainzgenre/')[-1]
            genre.append(temp_genre)
        print('Genre found')
    except:
        print('No genre found')
    
    # Use genre values to check if review is classical or opera. If yes, then dismis this review.   
    if 'classical' in genre:
        print('Classical review found. Dismissed!')
        print('')
        continue
    
    if 'opera' in genre:
        print('Opera review found. Dismissed!')
        print('')
        continue

    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #--------- Clean review text function ---------#
    
    class MLStripper(HTMLParser):
        def __init__(self):
            self.reset()
            self.strict = False
            self.convert_charrefs= True
            self.fed = []
        def handle_data(self, d):
            self.fed.append(d)
        def get_data(self):
            return ''.join(self.fed)

    def strip_tags(html):
        s = MLStripper()
        s.feed(html)
        return s.get_data()

    #  Get text & clean up
    try:
        clean_text = strip_tags(fields.get('body'))
        text = clean_text
        print('Text found')
    except:
        print('No text found')
    
    # Get teaser text & clean up
    try: 
        teaser_text = strip_tags(fields.get('trailText'))
        print('Teaser text found')
    except:
        print('No teaser text found')
    
    # Get headline
    try:
        headline = fields.get('headline')
        print('Headline found')
    except:
        print('No headline found')
    
    # Clean up headline
    try:
        # Use python split and capitalize to make clean headline
        headline = headline.split('review – ')[-1].capitalize()
        print('Headline cleaned up')
    except:
        print('Headline could not be cleaned')
    
    # Get Rating
    try:
        rating = fields.get('starRating')
        print('Rating found')
    except:
        print('No rating found')
    
    #--------- Scraping ---------#
    #--------- Get Venue ---------#
    # Get webURL
    url = response.get("response").get('content').get('webUrl')

    # Get the source code of the webpage as text
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')

    # Set the path and extract venue name
    locate_venue_path = soup.find("div", {"data-link-name": "standfirst"})
    try:
        venue_name = locate_venue_path.strong.text
    except:
        venue_name = ''
        print('could not find venue')

    # Seperate venue name from venue city
    try:
        venue_name = venue_name.split(',')[0].capitalize()
    except:
        print('No city added to venue name string')
    print(venue_name)
    
    # Get artist_name 
    try:
        artist_str = soup.title.getText()
        artist_str = re.search('\|(.*?)\|', artist_str).group(1)
        artist_str = artist_str.lstrip()
        artist_str = artist_str.rstrip()
        
        
        if artist_str.lower() == 'music':
            print('Artist name from title: ', artist_str)
            print('No artist name found')
            
        else:
            artist_name = artist_str
            print(artist_name)
            print('Artist name found')
        
    except:
        artist = ''
        print('Artist name not found')
    
    
    ########## Spotify lookup ############
    try:
        # Preprocessing: Remove all within () and punctuation, set lowercase
        preprocessed_name = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", artist_name)
        preprocessed_name = preprocessed_name.lower()
        preprocessed_name = preprocessed_name.translate(str.maketrans('', '', string.punctuation))
        print('Name preprocessing:', artist_name, '=' ,preprocessed_name)

        # Reset spotify values
        spotify_match = False
        artist_name_spotify = ""
        artist_img_spotify = ""
        artist_spotify_id = ""
        subgenres_spotify = []
        artist_popularity_spotify = ""

        # Make Spotify lookup on raw artist name
        results = spotify.search(q='artist:' + artist_name, type='artist')
        items = results['artists']['items']
        
        # If spotify returns no results on raw artist name search, make Spotify lookup on preprocessed artist name

        if len(items) == 0:
            print('No result on raw name, trying preprocessed name...')
            results = spotify.search(q='artist:' + preprocessed_name, type='artist')
            items = results['artists']['items']

        # If spotify returns results on raw artist name
        if len(items) > 0:

            print('found', len(items), 'results')
            print('searching for exact string match...')

            # First check all results for exact string match
            for ii in range(len(items)):
                artist = items[ii]

                # Set lower case and match songkick name with spotify name
                if artist_name.lower() == artist['name'].lower():

                    spotify_match = True
                    print('Found exact string match on index', ii, artist['name'])

                    artist_name_spotify = artist['name']
                    artist_img_spotify = artist['images'][0]['url']
                    artist_spotify_id = artist['id']
                    subgenres_spotify = artist['genres']
                    artist_popularity_spotify = artist['popularity']

                    break

                else:
                    print('No exact match on result from index ', ii, artist['name'])

            # If no match, then check all spotify results for fuzzy string match above 90%
            if spotify_match == False:
                print('searching for fuzzy string match...')

                for iii in range(len(items)):
                    artist = items[iii]

                    Distance = lev.distance(preprocessed_name,artist['name'].lower()),
                    Ratio = lev.ratio(preprocessed_name,artist['name'].lower())

                    if Ratio > 0.90:

                        spotify_match = True
                        print('Found fuzzy string match on index', iii, artist['name'])
                        print('with ratio:', Ratio)

                        artist_name_spotify = artist['name']
                        artist_img_spotify = artist['images'][0]['url']
                        artist_spotify_id = artist['id']
                        subgenres_spotify = artist['genres']
                        artist_popularity_spotify = artist['popularity']
                        break

                    else:
                        print('No fuzzy match on result from index ', iii, artist['name'])

        else:
            print('no Spotify search results at all from the preprocessed string')
    except:
        print('unsuccesful spotify lookup')

    
    #--------- Define all values for Tribute database ---------#
    
    print('')
    print('Critic:', critic_name)
    print('Author:', author)
    print('URL:', url)
    print('Artist:', artist_name)
    print('Venue:', venue_name)
    print('Venue Place:', venue_place)
    print('Rating:', str(rating))
    print('Max Rating:', str(max_rating))
    print('Publication Date:', publication_date)
    print('Concert Date:', concert_date)
    print('artist spotify name:', artist_name_spotify)
    print('artist spotify image:', artist_img_spotify)
    print('artist spotify ID:', artist_spotify_id)
    print('artist spotify subgenres:', subgenres_spotify)
    print('artist spotify popularity:', artist_popularity_spotify)
    print('Language of review:', lang)
    print('Headline:', headline)
    print('Text:', text)
    print('')

    
    # Set dictionary values

    temp_dict = {}
    temp_dict['critic_name'] = critic_name
    temp_dict['author'] = author
    temp_dict['url'] = url
    temp_dict['artist_name'] = artist_name
    temp_dict['artist_name_spotify'] = artist_name_spotify
    temp_dict['artist_img_spotify'] = artist_img_spotify
    temp_dict['artist_spotify_id'] = artist_spotify_id
    temp_dict['artist_popularity_spotify'] = artist_popularity_spotify
    temp_dict['genre'] = genre
    temp_dict['subgenre_spotify'] = subgenres_spotify
    temp_dict['publication_date'] = publication_date
    temp_dict['concert_date'] = concert_date
    temp_dict['venue_name'] = venue_name
    temp_dict['venue_place'] = venue_place
    temp_dict['rating'] = rating
    temp_dict['max_rating'] = max_rating
    temp_dict['review_language'] = lang
    temp_dict['headline'] = headline
    temp_dict['text'] = text

    critic_review_dict['concerts'].append(temp_dict)


Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Text found
Teaser text found
Headline found
Headline cleaned up
Rating found
Brighton dome
Artist name from title:  Music
No artist name found
Name preprocessing:  = 
found 10 results
searching for exact string match...
No exact match on result from index  0 Artist Vs Poet
No exact match on result from index  1 Artists Of Then, Now & Forever
No exact match on result from index  2 Artist Trei
No exact ma

Status Code: 200
Genre extracted from API
Genre found
Text found
Teaser text found
Headline found
Headline cleaned up
Rating found
Sage
Artist name from title:  Music
No artist name found
Name preprocessing:  = 
found 10 results
searching for exact string match...
No exact match on result from index  0 Artist Vs Poet
No exact match on result from index  1 Artists Of Then, Now & Forever
No exact match on result from index  2 Artist Trei
No exact match on result from index  3 Artists for Haiti
No exact match on result from index  4 Artist17
No exact match on result from index  5 Artists Stand Up To Cancer
No exact match on result from index  6 Artists for Peace and Justice
No exact match on result from index  7 Artist_lr
No exact match on result from index  8 Artist Unknown
No exact match on result from index  9 Artista Rosario
searching for fuzzy string match...
No fuzzy match on result from index  0 Artist Vs Poet
No fuzzy match on result from index  1 Artists Of Then, Now & Forever
No

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

Status Code: 200
Genre extracted from API
Genre found
Classical review found. Dismissed!

